<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter05/Detectron2_Chapter05_Hook.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 05-Hook
## The dataset
Should execute the Notebook titled "Data Processing" first to process the dataset or run the following code to download the processed dataset from GitHub repository of the book.

In [1]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

## Train models
Installation

In [2]:
!python -m pip install \
'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-dl1mzpup
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-dl1mzpup
     |████████████████████████████████| 50 kB 4.4 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 153 kB 75.8 MB/s 
     |████████████████████████████████| 1.5 MB 65.7 MB/s 
     |████████████████████████████████| 549 kB 67.9 MB/s 
     |████████████████████████████████| 117 kB 81.3 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
     |████████████████████████████████| 182 kB 80.3 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=5463860 sha256=600b62669b70ea6b068c13fa20a221a189883479a4bb83330c1113d5c4b4be49
  Stored in directory: /tmp/pip-ephem-wheel-cache-11is8z9m/wheels/19/ac/6

In [3]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

In [4]:
from detectron2.data.datasets import register_coco_instances

In [5]:
# Some configurations
name_ds = "braintumors_coco"
name_ds_train = name_ds + "_train"
name_ds_test = name_ds + "_test"
image_root_train = name_ds + "/train"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_train = name_ds + "/train/" + af
json_file_test = name_ds + "/test/" + af

In [6]:
# Register datasets
## train dataset
register_coco_instances(
    name = name_ds_train,
    metadata = {},
    json_file = json_file_train,
    image_root = image_root_train
    )
## test dataset
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )


### Training configuration

In [7]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
import pickle

In [8]:
output_dir = "output/object_detector_hook"
os.makedirs(output_dir, exist_ok=True)
output_cfg_path = os.path.join(output_dir, "cfg.pickle")
nc = 2
device = "cuda"
# Select a model
config_file_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [9]:
# Create a configuration file
cfg = get_cfg()
config_file = model_zoo.get_config_file(config_file_url)
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Download weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Set datasets
cfg.DATASETS.TRAIN = (name_ds_train,)
cfg.DATASETS.TEST = (name_ds_test,)
# Workers
cfg.DATALOADER.NUM_WORKERS = 2
# Images per batch
cfg.SOLVER.IMS_PER_BATCH = 8
# Learning rate
cfg.SOLVER.BASE_LR = 0.00025
# Iterations
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.CHECKPOINT_PERIOD = 500
# Evaluation
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.CHECKPOINT_PERIOD
# Classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = nc
cfg.MODEL.DEVICE = device
cfg.OUTPUT_DIR = output_dir

In [10]:
# save configuration file for future use
with open(output_cfg_path, "wb") as f:
  pickle.dump(cfg, f, protocol = pickle.HIGHEST_PROTOCOL)

### Training

In [11]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class BrainTumorTrainer(DefaultTrainer):
  """
  This trainer evaluate data on the `cfg.DATASETS.TEST` validation dataset every `cfg.TEST.EVAL_PERIOD` iterations.
  """
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder == None:
      output_folder = cfg.OUTPUT_DIR
    else:
      output_folder = os.path.join(cfg.OUTPUT_DIR, output_folder)
      os.makedirs(output_folder)
    # Use 
    return COCOEvaluator(dataset_name, distributed=False, output_dir=output_folder)


In [12]:
from detectron2.engine.hooks import HookBase
import torch
import logging

class BestModelHook(HookBase):
  def __init__(self, cfg, metric="bbox/AP50", min_max="max"):
    self._period = cfg.TEST.EVAL_PERIOD
    self.metric = metric
    self.min_max = min_max
    self.best_value = float("-inf") if min_max == "max" else float("inf")
    logger = logging.getLogger("detectron2")
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    self._logger = logger
        

  def _take_latest_metrics(self):
    with torch.no_grad():
      latest_metrics = self.trainer.storage.latest()
      return latest_metrics
      
  def after_step(self):
    next_iter = self.trainer.iter + 1
    is_final = next_iter == self.trainer.max_iter
    if is_final or (self._period > 0 and next_iter % self._period == 0):
      latest_metrics = self._take_latest_metrics()
      for (key, (value, iter)) in latest_metrics.items():
        if key == self.metric:
          if (self.min_max == "min" and value < self.best_value) or (self.min_max == "max" and value > self.best_value):
            self._logger.info("Updating best model at iteration {} with {} = {}".format(iter, self.metric, value))
            self.best_value = value
            self.trainer.checkpointer.save("model_best")
            

In [13]:
trainer = BrainTumorTrainer(cfg)
bm_hook = BestModelHook(cfg, metric="bbox/AP50", min_max="max")
trainer.register_hooks(hooks=[bm_hook])
trainer.train()

[12/20 20:31:58 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/20 20:32:39 d2.utils.events]:  eta: 2:03:46  iter: 19  total_loss: 25.19  loss_cls: 15.14  loss_box_reg: 3.714  loss_rpn_cls: 1.167  loss_rpn_loc: 2.286  time: 1.4500  data_time: 0.0818  lr: 4.9953e-06  max_mem: 9628M
[12/20 20:33:12 d2.utils.events]:  eta: 2:09:29  iter: 39  total_loss: 2.952  loss_cls: 1.521  loss_box_reg: 0.4114  loss_rpn_cls: 0.4071  loss_rpn_loc: 0.4503  time: 1.5498  data_time: 0.0710  lr: 9.9902e-06  max_mem: 9629M
[12/20 20:33:43 d2.utils.events]:  eta: 2:08:44  iter: 59  total_loss: 1.988  loss_cls: 1.335  loss_box_reg: 0.1459  loss_rpn_cls: 0.3707  loss_rpn_loc: 0.2635  time: 1.5433  data_time: 0.0699  lr: 1.4985e-05  max_mem: 9629M
[12/20 20:34:15 d2.utils.events]:  eta: 2:09:52  iter: 79  total_loss: 0.7849  loss_cls: 0.3353  loss_box_reg: 0.06439  loss_rpn_cls: 0.2139  loss_rpn_loc: 0.1618  time: 1.5686  data_time: 0.0732  lr: 1.998e-05  max_mem: 9629M
[12/20 20:34:50 d2.utils.events]:  eta: 2:13:04  iter: 99  total_loss: 0.5076  loss_cls: 0.1702  loss

## Save training information

In [14]:
# remove the .pth file (do not run this if you would like to keep the models)
!rm {cfg.OUTPUT_DIR}/*.pth
# zip
!zip -r {cfg.OUTPUT_DIR}.zip {cfg.OUTPUT_DIR}
from google.colab import files
files.download(cfg.OUTPUT_DIR+".zip")

  adding: output/object_detector_hook/ (stored 0%)
  adding: output/object_detector_hook/events.out.tfevents.1671568320.7e4f25809f8c.132.0 (deflated 72%)
  adding: output/object_detector_hook/coco_instances_results.json (deflated 71%)
  adding: output/object_detector_hook/cfg.pickle (deflated 50%)
  adding: output/object_detector_hook/metrics.json (deflated 78%)
  adding: output/object_detector_hook/last_checkpoint (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>